# OpenAI Assistants APIs

The Assistants' API lets you create AI assistants in your applications. These assistants follow instruction. They use models, tools, and knowledge to answer user questions. In this notebook we are going to use one of the tools, retriever, to query against two pdf documents we will upload.

The architecture and data flow diagram below depicts the interaction among all components that comprise OpenAI Assistant APIs. Central to understand is the Threads and Runtime that executes asynchronously, adding and reading messages to the Threads.

For integrating the Assistants API:

1. Creat an Assistant with custom instructions and select a model. Optionally, enable tools like Code Interpreter, Retrieval, and Function Calling.

2. Initiate a Thread for each user conversation.
    
3. Add user queries as Messages to the Thread.

4.  Run the Assistant on the Thread for responses, which automatically utilizes the enabled tools

Below we follow those steps to demonstrate how to integrate Assistants API, using Retrieval tool, to a) upload a couple of pdf documents and b) use Assistant to query the contents of the document. Consider this as a mini Retrieval Augmented Generation (RAG). 

The OpenAI documentation describes in details [how Assistants work](https://platform.openai.com/docs/assistants/how-it-works).

<img src="./images/assistant_ai_tools_retriever.png">


## How to use Assistant API using Tools: Retriever

In [1]:
import warnings
import os
import time

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
from typing import List
from assistant_utils import print_thread_messages, upload_files, \
                            loop_until_completed, create_assistant_run

Load our .env file with respective API keys and base url endpoints. Here you can either use OpenAI or Anyscale Endpoints. **Note**: Assistant API calling for Anyscale Endpoints (which serves only OS modles) is not yet aviable).

In [2]:
warnings.filterwarnings('ignore')

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
DOCS_TO_LOAD = ["docs/HAI_AI-Index-Report_2023.pdf"]

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

### Step 1: Create our knowledgebase
This entails uploading your pdfs as your knowledgebase for the retrievers to use. Once you upload a file, the Assistant from OpenAI will break it into smaller chuncks, sort and save these chuncks, index and store the embeddings as vectors. 

The retrievers use your query to retrieve the best semantic matches on vectors in the knowledgebase, and then feed the LLM, along with the original query, to generate the consolidated and comprehesive answer, similarly to how a large-scale RAG retriever operates.

In [5]:
file_objects = upload_files(client, DOCS_TO_LOAD)
file_objects

[FileObject(id='file-ibaAJOuZhpjf2Yw6e0RS8119', bytes=25318310, created_at=1704921232, filename='HAI_AI-Index-Report_2023.pdf', object='file', purpose='assistants', status='processed', status_details=None)]

In [6]:
# Extract file ids 
file_obj_ids = []
for f_obj in file_objects:
    file_obj_ids.append(file_objects[0].id)
file_obj_ids

['file-ibaAJOuZhpjf2Yw6e0RS8119']

### Step 2: Create an Assistant 
Before you can start interacting with the Assistant to carry out any tasks, you need an AI assistant object. Supply the Assistant with a model to use, tools, and file ids to use for its knowledge base.

In [7]:
assistant = client.beta.assistants.create(name="AI Report and LLM survey Chatbot",
                                           instructions="""You are a knowledgeable chatbot trained to respond 
                                               inquires on documents HAI Artificial Index 2023 report 
                                               and Survey of why LLMs hallucinate. 
                                               Use a neutral, professional advisory tone, and only respond by consulting the 
                                               knowledge base or files you are granted access to. 
                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid
                                               I don't have access to that information.'""",
                                           model=MODEL,
                                           tools = [{'type': 'retrieval'}],  # use the retrieval tool
                                           file_ids=file_obj_ids # use these files uploaded as part of your knowledge base
)                                        
assistant

Assistant(id='asst_waMgBV2L5xMldUkU5pO5m2Ur', created_at=1704921233, description=None, file_ids=['file-ibaAJOuZhpjf2Yw6e0RS8119'], instructions="You are a knowledgeable chatbot trained to respond \n                                               inquires on documents HAI Artificial Index 2023 report \n                                               and Survey of why LLMs hallucinate. \n                                               Use a neutral, professional advisory tone, and only respond by consulting the \n                                               knowledge base or files you are granted access to. \n                                               Do not make up answers. If you don't know answer, respond with 'Sorry, I'm afraid\n                                               I don't have access to that information.'", metadata={}, model='gpt-4-1106-preview', name='AI Report and LLM survey Chatbot', object='assistant', tools=[ToolRetrieval(type='retrieval')])

### Step 3: Create a thread 
As the diagram above shows, the Thread is the object with which the AI Assistant runs will interact with, by fetching messages and putting messages to it. Think of a thread as a "conversation session between an Assistant and a user. Threads store Messages and automatically handle truncation to fit content into a model’s context window."

In [8]:
thread = client.beta.threads.create()
thread

Thread(id='thread_UF5gGietzOetVibpF7GU2ku7', created_at=1704921243, metadata={}, object='thread')

### Step 4: Add your message query to the thread for the Assistant

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.
    Summarize each takeway in no more three simple sentences.""",
)
message.model_dump()

{'id': 'msg_PSFQObvoEWY9C7OAREYWVHca',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'What are the top 10 takeaways in the Artificial Intelligence Index Report 2023.\n    Summarize each takeway in no more three simple sentences.'},
   'type': 'text'}],
 'created_at': 1704921244,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_UF5gGietzOetVibpF7GU2ku7'}

### Step 5: Create a Run for the Assistant
A Run is an invocation of an Assistant on a Thread. The Assistant uses its configuration and the Thread’s Messages to perform tasks by calling models and tools. As part of a Run, the Assistant appends Messages to the Thread.

Note that Assistance will run asychronously: the run has the following
lifecycle and states: [*expired, completed, failed, cancelled*]. Run objects can have multiple statuses.

<img src="https://cdn.openai.com/API/docs/images/diagram-1.png">

In [10]:
instruction_msg = """Please address the user as Jules Dmatrix.  
    Do not provide an answer to the question if the information was not retrieved from the knowledge base.
"""
run = create_assistant_run(client, assistant, thread, instruction_msg)
print(run.model_dump_json(indent=4))

{
    "id": "run_B7AsSLvwUmhlTsgumunDmvqo",
    "assistant_id": "asst_waMgBV2L5xMldUkU5pO5m2Ur",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1704921244,
    "expires_at": 1704921844,
    "failed_at": null,
    "file_ids": [
        "file-ibaAJOuZhpjf2Yw6e0RS8119"
    ],
    "instructions": "Please address the user as Jules Dmatrix.  \n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_UF5gGietzOetVibpF7GU2ku7",
    "tools": [
        {
            "type": "retrieval"
        }
    ]
}


### Step 6: Loop through the Assistant run until status is 'completed'

In [11]:
run_status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)
print(run_status.model_dump_json(indent=4))

{
    "id": "run_B7AsSLvwUmhlTsgumunDmvqo",
    "assistant_id": "asst_waMgBV2L5xMldUkU5pO5m2Ur",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1704921244,
    "expires_at": 1704921844,
    "failed_at": null,
    "file_ids": [
        "file-ibaAJOuZhpjf2Yw6e0RS8119"
    ],
    "instructions": "Please address the user as Jules Dmatrix.  \n    Do not provide an answer to the question if the information was not retrieved from the knowledge base.\n",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-1106-preview",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_UF5gGietzOetVibpF7GU2ku7",
    "tools": [
        {
            "type": "retrieval"
        }
    ]
}


#### Poll until Assistant run is completed

In [12]:
loop_until_completed(client, thread, run_status)

in_progress
in_progress
in_progress
completed


### Step 7: Retrieve the message returned by the assistance
Only when the run is **completed** can you fetch the messages from the Thread

In [20]:
print_thread_messages(client, thread)

('assistant:Chapter 8 of the Artificial Intelligence Index Report 2023 focuses '
 'on the various opinions held by the public regarding AI. The chapter '
 'highlights that monitoring public attitudes is crucial as AI technology '
 "could significantly influence society's future. Surveys from multiple "
 'sources reveal differing global, national, and demographic perspectives on '
 'AI, lack of longitudinal data for AI-related questions, and varying '
 'sentiments among AI researchers. Social media discussions on AI are also '
 'examined in a bid to understand the broader public conversation about AI. It '
 'is noted that as AI continues to permeate daily life, comprehensive efforts '
 'to gauge public opinion on it will become even more significant.')
('user:Provide a short overview of Chatper 8 on public opinion in no more '
 'than\n'
 '    five sentences\n'
 '    ')
('assistant:Here are the top ten takeaways from the Artificial Intelligence '
 'Index Report 2023, summarized:\n'
 '\n'

### Repeat the process for any additional messages
To add more query messages to the thread for the Assistant,
repeat steps 5 - 7

### Add another message to for the Assistant

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user",
    content="""Provide a short overview of Chatper 8 on public opinion in no more than
    five sentences
    """,
)
message

ThreadMessage(id='msg_vv58iPrQJDspKdxn1kwo8B0t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Provide a short overview of Chatper 8 on public opinion in no more than\n    five sentences\n    '), type='text')], created_at=1704921285, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_UF5gGietzOetVibpF7GU2ku7')

### Create another run for the Assistant for the second message

In [15]:
run = create_assistant_run(client, assistant, thread, instruction_msg)

In [16]:
run_status = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

print(run_status.status)

queued


In [17]:
loop_until_completed(client, thread, run_status)

queued
in_progress
completed


In [18]:
print_thread_messages(client, thread)

('assistant:Chapter 8 of the Artificial Intelligence Index Report 2023 focuses '
 'on the various opinions held by the public regarding AI. The chapter '
 'highlights that monitoring public attitudes is crucial as AI technology '
 "could significantly influence society's future. Surveys from multiple "
 'sources reveal differing global, national, and demographic perspectives on '
 'AI, lack of longitudinal data for AI-related questions, and varying '
 'sentiments among AI researchers. Social media discussions on AI are also '
 'examined in a bid to understand the broader public conversation about AI. It '
 'is noted that as AI continues to permeate daily life, comprehensive efforts '
 'to gauge public opinion on it will become even more significant.')
('user:Provide a short overview of Chatper 8 on public opinion in no more '
 'than\n'
 '    five sentences\n'
 '    ')
('assistant:Here are the top ten takeaways from the Artificial Intelligence '
 'Index Report 2023, summarized:\n'
 '\n'

In [19]:
# Delete the assistant. Optionally, you can delete any files
# associated with it that you have uploaded onto the OpenAI platform

response = client.beta.assistants.delete(assistant.id)
print(response)

for file_id in file_obj_ids:
    print(f"deleting file id: {file_id}...")
    response = client.files.delete(file_id)
    print(response)

AssistantDeleted(id='asst_waMgBV2L5xMldUkU5pO5m2Ur', deleted=True, object='assistant.deleted')
deleting file id: file-ibaAJOuZhpjf2Yw6e0RS8119...
FileDeleted(id='file-ibaAJOuZhpjf2Yw6e0RS8119', deleted=True, object='file')
